In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
from openai import AsyncOpenAI
# import sendgrid
import os
# from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
import requests
import resend

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


In [2]:
load_dotenv(override=True)
RESEND_API_KEY = os.getenv("RESEND_API_KEY")

##Agent Workflow

In [3]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [4]:
BASE_URL = "https://api.groq.com/openai/v1"
MODEL_NAME='llama-3.3-70b-versatile'

In [5]:
groq_client = AsyncOpenAI(base_url=BASE_URL, api_key=os.getenv("GROQ_API_KEY"))

custom_model = OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=groq_client)

In [6]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=custom_model
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=custom_model
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=custom_model
)

In [7]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC2 Compliance with ComplAI

Dear [Recipient's Name],

I came across your company, [Company Name], and was impressed by the innovative work you're doing in [Industry/Field]. As a leader in the industry, I'm sure you understand the importance of maintaining the highest level of security and compliance, particularly when it comes to SOC2.

At ComplAI, we've developed an AI-powered SaaS tool designed to simplify and accelerate the SOC2 compliance process. Our platform helps organizations like yours efficiently prepare for audits, identify potential risks, and demonstrate compliance with ease.

With ComplAI, you'll be able to:

* Automate compliance workflows and reduce manual effort
* Identify and remediate control gaps with our AI-driven risk assessment
* Streamline audit preparation and reduce the time spent on audit-related activities
* Enhance visibility and transparency into your compliance posture

Our solution is designed to help you achieve and maintain S

In [8]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Subject: Streamline Your SOC 2 Compliance with ComplAI

Dear [Recipient's Name],

I came across your company, [Company Name], and was impressed by the innovative work you're doing in [industry/field]. As a leader in the [industry/field] space, I'm sure you understand the importance of maintaining the highest standards of security and compliance.

As you continue to grow and scale, ensuring SOC 2 compliance is crucial to building trust with your customers and stakeholders. However, the process of preparing for a SOC 2 audit can be time-consuming, complex, and resource-intensive.

That's where ComplAI can help. Our cutting-edge SaaS tool, powered by AI, simplifies the SOC 2 compliance process, helping you prepare for audits with ease and confidence. With ComplAI, you can:

* Automate compliance monitoring and reporting
* Identify and mitigate risks in real-time
* Streamline audit preparation and reduce costs
* Ensure continuous compliance with evolving regulations

Our AI-driven platform

In [9]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model=custom_model
)

In [10]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: Simplify SOC2 Compliance with ComplAI

Dear [Recipient's Name],

I hope this email finds you well. As a [Recipient's Title] at [Company Name], I'm sure you understand the importance of maintaining SOC2 compliance in today's complex regulatory landscape. Achieving and maintaining this standard can be a daunting task, requiring significant time, resources, and expertise.

At ComplAI, we believe that compliance should be a catalyst for growth, not a hindrance. Our innovative SaaS tool, powered by AI, is designed to streamline the SOC2 compliance process, reducing the burden on your team and ensuring a seamless audit experience.

With ComplAI, you'll be able to:

* Automate control monitoring and evidence collection
* Identify and mitigate potential risks and gaps in real-time
* Generate audit-ready reports and documentation
* Collaborate with your team and stakeholders in a centralized platform

Our solution is designed to help you navigate the complexities of S

## Use of Tools

In [11]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=custom_model
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=custom_model
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=custom_model
)

In [12]:
sales_agent1

Agent(name='Professional Sales Agent', instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x10bce2030>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

## Tools and Agent integration

In [18]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects via Resend """
    
    # Set up email sender, recipient, and content
    from_email = "shardulggore@gmail.com"  # Replace with your verified sender
    to_email = "shardulgg972001@gmail.com"  # Replace with recipient's email
    
    # Resend API headers and payload
    headers = {
        "Authorization": f"Bearer {RESEND_API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "from": f"onboarding@resend.dev",
        "to": "shardulggore@gmail.com",
        "subject": "Sales email",
        "html": f"<p>{body}</p>"  # Body wrapped in <p> tags for HTML format
    }
    
    # Send email using Resend API
    response = requests.post("https://api.resend.com/emails", json=payload, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 202:
        return {"status": "success"}
    else:
        return {"status": "failure", "message": response.text}

In [19]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects via Resend', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10c3deca0>, strict_json_schema=True)

### Convert agent to tool

In [23]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10cba2b60>, strict_json_schema=True)

In [24]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10cba28e0>, strict_json_schema=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x10b1465c0>, strict_json_schema=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent

In [25]:
instructions ="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model=custom_model)

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)